# ARI with sweeping dimension reduction components and bin size of MICA MDS mode

In [1]:
import sys, os
import glob
import shutil

import pandas as pd
import numpy as np
import anndata as ad

from sklearn.metrics import (silhouette_score, silhouette_samples,   
                             adjusted_rand_score, adjusted_mutual_info_score,   
                             normalized_mutual_info_score)
pd.set_option('display.max_colwidth', None)

In [2]:
pred_label_root = './'
true_label_root = '../../../../Dataset/scMINER_13datasets_h5ad_input_true_label/'

In [3]:
def eval_pred_label(true_label_df,pred_files,component,bin_size):
    eval_df = []
    for each_file in pred_files:        
        pred_df = pd.read_table(each_file)

        #merge_df = pred_df.merge(true_label_df,left_on = 'ID',right_on='cell', how='inner') 
        merge_df = pd.concat([pred_df,true_label_df],axis=1) 
        #print(merge_df)
        #break
    
        pred_k = len(set(merge_df['label']))
        #print(pred_k)
        #break
        ARI = np.round(adjusted_rand_score(merge_df['true_label'],merge_df['label']),3)

        eval_df.append([component,bin_size,pred_k,ARI])
    
    return eval_df

In [4]:
for folder in glob.glob(pred_label_root+'*/'):
    dataset = folder.split('/')[1]
    #print(dataset)

    # true label
    true_label_file = glob.glob(true_label_root + dataset+ '*true_label.txt')[0]
    #print(true_label_file)
    true_label_df = pd.read_table(true_label_file)

    summary_df = []
    for folder in glob.glob(pred_label_root + dataset + '/'+ dataset + '*/'):
        #print(folder)
        
        bin_size = int(folder.split('_')[-1][:-1])
        component = int(folder.split('_')[-3])
        #break
        #

        #print(component)
        #break
        pred_files = glob.glob(folder + 'clustering*.txt')
        #print(pred_files)
        #break
        eval_df = eval_pred_label(true_label_df,pred_files,component,bin_size)
        #eval_df = pd.DataFrame(eval_df)
        #print(eval_df)
        
        summary_df.extend(eval_df)
    
    summary_df = pd.DataFrame(summary_df)
    summary_df.rename(columns = {0:'component',1:'bin_size',2:'pred_k',3:'ARI'},inplace=True)
    summary_df.to_csv(dataset + '_scMINER_sweep_component_binsize_summary.csv',sep='\t')

    #break

# Check Kolod ARI

In [17]:
#Kolod_report_file = '../../../../scMINER_clustering_evaluation/Clustering_outputs_best_k/Clustering_outputs/scMINER/Kolod_k3_ARI_1.txt'
Kolod_report_file = '/Volumes/xyao/MICA/mica_100/bin_size_dimension_sweeping/MDS_bin_size_dimension/Kolod/Kolod_component_96_binsize_95/clustering_UMAP_euclidean_96_3.txt'
df = pd.read_table(Kolod_report_file)
df

,ID,X,Y,label
0,1,5.790238,4.761262,3
1,2,6.313134,4.556213,3
2,3,7.553454,5.897995,3
3,4,7.824313,6.050239,3
4,5,6.903933,4.425219,3
...,...,...,...,...
699,700,4.344112,-11.010385,1
700,701,5.330587,-9.518352,1
701,702,4.620634,-10.671297,1
702,703,4.217658,-10.889918,1


In [18]:
Kolod_true_label = '../../../../Dataset/scMINER_13datasets_h5ad_input_true_label/Kolod_true_label.txt'
true_label_df = pd.read_table(Kolod_true_label)
true_label_df

,cell,true_label
0,ola_mES_2i_2_1s,1
1,ola_mES_2i_2_10s,1
2,ola_mES_2i_2_11s,1
3,ola_mES_2i_2_12s,1
4,ola_mES_2i_2_13s,1
...,...,...
699,ola_mES_lif_3_90s,3
700,ola_mES_lif_3_92s,3
701,ola_mES_lif_3_94s,3
702,ola_mES_lif_3_95s,3


In [19]:
#merge_df = df.merge(true_label_df,left_on = 'ID',right_on='cell', how='inner') 
merge_df = pd.concat([df,true_label_df],axis=1) 
merge_df

,ID,X,Y,label,cell,true_label
0,1,5.790238,4.761262,3,ola_mES_2i_2_1s,1
1,2,6.313134,4.556213,3,ola_mES_2i_2_10s,1
2,3,7.553454,5.897995,3,ola_mES_2i_2_11s,1
3,4,7.824313,6.050239,3,ola_mES_2i_2_12s,1
4,5,6.903933,4.425219,3,ola_mES_2i_2_13s,1
...,...,...,...,...,...,...
699,700,4.344112,-11.010385,1,ola_mES_lif_3_90s,3
700,701,5.330587,-9.518352,1,ola_mES_lif_3_92s,3
701,702,4.620634,-10.671297,1,ola_mES_lif_3_94s,3
702,703,4.217658,-10.889918,1,ola_mES_lif_3_95s,3


In [20]:
np.round(adjusted_rand_score(merge_df['true_label'],merge_df['label']),3)

1.0

# Kolod ARI not affected by bin size nor dimension

# Check Ding Liang folder

In [7]:
Kolod_report_file = '/Volumes/projects/scRNASeq/yu3grp/LiangDing/MICA/outputs/GoldenStd/Kolod/mds_1_3/Kolod_k3_umap_ClusterMem.txt'
df = pd.read_table(Kolod_report_file)
df

,Unnamed: 0,X,Y,label
0,1,17.030450,5.710739,1
1,2,20.245396,2.663836,1
2,3,20.232512,2.388761,1
3,4,17.558315,4.758851,1
4,5,17.506480,4.134029,1
...,...,...,...,...
699,700,6.028198,-1.708858,2
700,701,6.216826,-2.200513,2
701,702,6.596740,-2.647401,2
702,703,5.990533,-3.193825,2


In [8]:
# Assume cell order is consistent
merge_df = pd.concat([df,true_label_df],axis=1) 
merge_df

,Unnamed: 0,X,Y,label,cell,true_label
0,1,17.030450,5.710739,1,ola_mES_2i_2_1s,1
1,2,20.245396,2.663836,1,ola_mES_2i_2_10s,1
2,3,20.232512,2.388761,1,ola_mES_2i_2_11s,1
3,4,17.558315,4.758851,1,ola_mES_2i_2_12s,1
4,5,17.506480,4.134029,1,ola_mES_2i_2_13s,1
...,...,...,...,...,...,...
699,700,6.028198,-1.708858,2,ola_mES_lif_3_90s,3
700,701,6.216826,-2.200513,2,ola_mES_lif_3_92s,3
701,702,6.596740,-2.647401,2,ola_mES_lif_3_94s,3
702,703,5.990533,-3.193825,2,ola_mES_lif_3_95s,3


In [9]:
np.round(adjusted_rand_score(merge_df['true_label'],merge_df['label']),3)

1.0

In [10]:
Kolod_report_file = '/Volumes/projects/scRNASeq/yu3grp/LiangDing/MICA/outputs/GoldenStd/Kolod/mds_1_3_silhouette/clustering_umap_euclidean_19.txt'
df = pd.read_table(Kolod_report_file)
df

,Unnamed: 0,X,Y,label
0,1,7.481912,-3.299610,1
1,2,7.877691,-0.395978,1
2,3,7.777650,-0.490885,1
3,4,7.538467,-2.514733,1
4,5,7.609655,-2.192604,1
...,...,...,...,...
699,700,-5.417287,-1.822423,2
700,701,-5.612702,-2.375146,2
701,702,-5.541344,-2.519862,2
702,703,-6.028103,-2.662088,2


In [11]:
# Assume cell order is consistent
merge_df = pd.concat([df,true_label_df],axis=1) 
merge_df

,Unnamed: 0,X,Y,label,cell,true_label
0,1,7.481912,-3.299610,1,ola_mES_2i_2_1s,1
1,2,7.877691,-0.395978,1,ola_mES_2i_2_10s,1
2,3,7.777650,-0.490885,1,ola_mES_2i_2_11s,1
3,4,7.538467,-2.514733,1,ola_mES_2i_2_12s,1
4,5,7.609655,-2.192604,1,ola_mES_2i_2_13s,1
...,...,...,...,...,...,...
699,700,-5.417287,-1.822423,2,ola_mES_lif_3_90s,3
700,701,-5.612702,-2.375146,2,ola_mES_lif_3_92s,3
701,702,-5.541344,-2.519862,2,ola_mES_lif_3_94s,3
702,703,-6.028103,-2.662088,2,ola_mES_lif_3_95s,3


In [12]:
np.round(adjusted_rand_score(merge_df['true_label'],merge_df['label']),3)

1.0

# Check SC3s HVG2000

In [13]:
Kolod_report_file = '/Volumes/xyao/SC3s/scMINER_13datasets_batch_run_SC3s/Kolod/SC3s_clustering_sc3s_3_0.12.txt'
df = pd.read_table(Kolod_report_file)
df

,Unnamed: 0,cell_id,UMAP_X,UMAP_Y,SC3s_label,Sil_index
0,0,ola_mES_2i_2_1s,9.959663,14.530657,1,-0.128011
1,1,ola_mES_2i_2_10s,10.376685,13.696595,1,-0.034271
2,2,ola_mES_2i_2_11s,9.853083,13.629468,1,-0.048155
3,3,ola_mES_2i_2_12s,9.357112,14.521897,1,-0.066828
4,4,ola_mES_2i_2_13s,9.677513,15.153088,1,-0.039344
...,...,...,...,...,...,...
699,699,ola_mES_lif_3_90s,-10.702456,8.577251,0,0.071984
700,700,ola_mES_lif_3_92s,-9.936067,8.308652,0,0.138342
701,701,ola_mES_lif_3_94s,-9.841066,9.127931,0,0.165427
702,702,ola_mES_lif_3_95s,-9.111523,8.553529,0,0.147250


In [14]:
# Assume cell order is consistent
merge_df = pd.concat([df,true_label_df],axis=1) 
merge_df

,Unnamed: 0,cell_id,UMAP_X,UMAP_Y,SC3s_label,Sil_index,cell,true_label
0,0,ola_mES_2i_2_1s,9.959663,14.530657,1,-0.128011,ola_mES_2i_2_1s,1
1,1,ola_mES_2i_2_10s,10.376685,13.696595,1,-0.034271,ola_mES_2i_2_10s,1
2,2,ola_mES_2i_2_11s,9.853083,13.629468,1,-0.048155,ola_mES_2i_2_11s,1
3,3,ola_mES_2i_2_12s,9.357112,14.521897,1,-0.066828,ola_mES_2i_2_12s,1
4,4,ola_mES_2i_2_13s,9.677513,15.153088,1,-0.039344,ola_mES_2i_2_13s,1
...,...,...,...,...,...,...,...,...
699,699,ola_mES_lif_3_90s,-10.702456,8.577251,0,0.071984,ola_mES_lif_3_90s,3
700,700,ola_mES_lif_3_92s,-9.936067,8.308652,0,0.138342,ola_mES_lif_3_92s,3
701,701,ola_mES_lif_3_94s,-9.841066,9.127931,0,0.165427,ola_mES_lif_3_94s,3
702,702,ola_mES_lif_3_95s,-9.111523,8.553529,0,0.147250,ola_mES_lif_3_95s,3


In [16]:
np.round(adjusted_rand_score(merge_df['true_label'],merge_df['SC3s_label']),3)

0.98

In [22]:
Kolod_report_file = '/Volumes/xyao/Seurat/scMINER_13datasets_batch_run_Seurat/Batch_true_k/Kolod/Seurat_clustering_louvain_0.02_k3.txt'
df = pd.read_table(Kolod_report_file)
df

,cell_id,UMAP_X,UMAP_Y,Seurat_label
ola_mES_2i_2_1s,ola_mES_2i_2_1s,-5.664809,2.767055,1
ola_mES_2i_2_10s,ola_mES_2i_2_10s,-4.959332,3.423644,1
ola_mES_2i_2_11s,ola_mES_2i_2_11s,-5.268694,3.539542,1
ola_mES_2i_2_12s,ola_mES_2i_2_12s,-4.925235,3.883610,1
ola_mES_2i_2_13s,ola_mES_2i_2_13s,-4.924775,4.218164,1
...,...,...,...,...
ola_mES_lif_3_90s,ola_mES_lif_3_90s,6.998630,-2.421496,2
ola_mES_lif_3_92s,ola_mES_lif_3_92s,7.190856,-2.454479,2
ola_mES_lif_3_94s,ola_mES_lif_3_94s,7.563043,-2.608642,2
ola_mES_lif_3_95s,ola_mES_lif_3_95s,8.048380,-3.219336,2


In [25]:
# Assume cell order is consistent
merge_df = df.merge(true_label_df,left_on='cell_id',right_on='cell',how='inner')
merge_df

,cell_id,UMAP_X,UMAP_Y,Seurat_label,cell,true_label
0,ola_mES_2i_2_1s,-5.664809,2.767055,1,ola_mES_2i_2_1s,1
1,ola_mES_2i_2_10s,-4.959332,3.423644,1,ola_mES_2i_2_10s,1
2,ola_mES_2i_2_11s,-5.268694,3.539542,1,ola_mES_2i_2_11s,1
3,ola_mES_2i_2_12s,-4.925235,3.883610,1,ola_mES_2i_2_12s,1
4,ola_mES_2i_2_13s,-4.924775,4.218164,1,ola_mES_2i_2_13s,1
...,...,...,...,...,...,...
699,ola_mES_lif_3_90s,6.998630,-2.421496,2,ola_mES_lif_3_90s,3
700,ola_mES_lif_3_92s,7.190856,-2.454479,2,ola_mES_lif_3_92s,3
701,ola_mES_lif_3_94s,7.563043,-2.608642,2,ola_mES_lif_3_94s,3
702,ola_mES_lif_3_95s,8.048380,-3.219336,2,ola_mES_lif_3_95s,3


In [26]:
np.round(adjusted_rand_score(merge_df['true_label'],merge_df['Seurat_label']),3)

0.992

# check MICA input

# Check MICA batch run MDS

In [15]:
Kolod_report_file = '../../../../MICA/scMINER_13datasets_batch_run_MICA/Kolod/clustering_umap_euclidean_19.txt'
df = pd.read_table(Kolod_report_file)
df

,ID,X,Y,label
0,ola_mES_2i_2_1s,-8.202704,-1.140037,2
1,ola_mES_2i_2_10s,-7.185316,-2.237247,2
2,ola_mES_2i_2_11s,-7.282990,-2.375328,2
3,ola_mES_2i_2_12s,-7.073586,-0.356506,2
4,ola_mES_2i_2_13s,-7.677546,-0.174534,2
...,...,...,...,...
699,ola_mES_lif_3_90s,13.532821,5.463416,1
700,ola_mES_lif_3_92s,13.152828,5.792397,1
701,ola_mES_lif_3_94s,13.615450,5.900942,1
702,ola_mES_lif_3_95s,13.018428,6.397937,1


In [16]:
merge_df = df.merge(true_label_df,left_on = 'ID',right_on='cell', how='inner') 
merge_df

,ID,X,Y,label,cell,true_label
0,ola_mES_2i_2_1s,-8.202704,-1.140037,2,ola_mES_2i_2_1s,1
1,ola_mES_2i_2_10s,-7.185316,-2.237247,2,ola_mES_2i_2_10s,1
2,ola_mES_2i_2_11s,-7.282990,-2.375328,2,ola_mES_2i_2_11s,1
3,ola_mES_2i_2_12s,-7.073586,-0.356506,2,ola_mES_2i_2_12s,1
4,ola_mES_2i_2_13s,-7.677546,-0.174534,2,ola_mES_2i_2_13s,1
...,...,...,...,...,...,...
699,ola_mES_lif_3_90s,13.532821,5.463416,1,ola_mES_lif_3_90s,3
700,ola_mES_lif_3_92s,13.152828,5.792397,1,ola_mES_lif_3_92s,3
701,ola_mES_lif_3_94s,13.615450,5.900942,1,ola_mES_lif_3_94s,3
702,ola_mES_lif_3_95s,13.018428,6.397937,1,ola_mES_lif_3_95s,3


In [17]:
np.round(adjusted_rand_score(merge_df['true_label'],merge_df['label']),3)

0.727

In [18]:
Kolod_MICA_input = '/Volumes/projects/scRNASeq/yu3grp/scMINER/NG_Revision/XYao/MICA_Dataset/MICA_log2_txt_input/Kolod_MICA.log2.txt'
df = pd.read_table(Kolod_MICA_input)
df

,Unnamed: 0,ENSMUSG00000000001,ENSMUSG00000000028,ENSMUSG00000000037,ENSMUSG00000000056,ENSMUSG00000000058,ENSMUSG00000000078,ENSMUSG00000000085,ENSMUSG00000000088,ENSMUSG00000000094,...,ENSMUSG00000098211,ENSMUSG00000098217,ENSMUSG00000098219,ENSMUSG00000098222,ENSMUSG00000098224,ENSMUSG00000098232,ENSMUSG00000098234,ENSMUSG00000098235,ENSMUSG00000098238,ENSMUSG00000098240
0,ola_mES_2i_2_1s,6.604476,6.599217,0.0,2.736634,0.000000,7.523991,0.000000,8.728311,0.000000,...,0.0,0.772428,1.272790,1.044193,1.272790,1.938312,6.551002,0.000000,2.065793,3.780998
1,ola_mES_2i_2_10s,6.146365,7.551052,0.0,0.000000,0.000000,8.240065,0.250677,8.236439,0.000000,...,0.0,0.650118,0.814807,1.534887,1.870550,2.261625,6.617063,0.000000,1.943541,3.515651
2,ola_mES_2i_2_11s,7.761644,7.565213,0.0,0.000000,0.000000,7.026434,6.602249,7.807084,0.000000,...,0.0,1.040312,1.327418,1.237918,0.930336,0.380626,6.042423,0.380626,0.930336,3.512956
3,ola_mES_2i_2_12s,8.367752,7.161749,0.0,0.000000,0.000000,3.547153,0.000000,8.116923,0.000000,...,0.0,0.370572,0.525366,0.792575,1.608837,1.118667,6.741700,0.000000,1.974036,3.651705
4,ola_mES_2i_2_13s,6.618116,8.185765,0.0,0.000000,0.000000,6.957035,1.468765,8.290717,0.000000,...,0.0,0.813647,0.589859,1.330729,1.330729,1.007338,5.561222,0.324868,1.710590,3.896294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,ola_mES_lif_3_90s,8.842471,5.531304,0.0,0.455104,7.842117,10.402590,6.985600,6.050395,0.000000,...,0.0,1.079050,1.689428,3.307596,3.195252,2.446409,4.394212,0.455104,3.135618,3.411821
700,ola_mES_lif_3_92s,7.327939,6.320029,0.0,4.394841,0.000000,8.590420,2.979593,6.060602,0.000000,...,0.0,0.000000,2.046402,2.730171,2.428422,2.587171,7.383095,2.428422,2.587171,3.541581
701,ola_mES_lif_3_94s,9.534857,5.319054,0.0,0.000000,0.000000,9.611030,2.085209,5.090952,2.085209,...,0.0,1.845869,2.470072,2.470072,2.085209,2.085209,6.918918,0.000000,2.773587,3.237679
702,ola_mES_lif_3_95s,8.695979,6.706378,0.0,0.000000,0.000000,7.498861,0.000000,5.893844,0.000000,...,0.0,0.715819,2.074533,2.279321,3.011956,1.549545,7.359079,0.000000,2.279321,3.122506


In [27]:
Kolod_MICA_input = '../../../../MICA/scMINER_13datasets_batch_run_MICA/Kolod/Kolod_MICA.log2.txt'
df = pd.read_table(Kolod_MICA_input)
df

,Unnamed: 0,ENSMUSG00000000001,ENSMUSG00000000028,ENSMUSG00000000037,ENSMUSG00000000056,ENSMUSG00000000058,ENSMUSG00000000078,ENSMUSG00000000085,ENSMUSG00000000088,ENSMUSG00000000094,...,ENSMUSG00000098211,ENSMUSG00000098217,ENSMUSG00000098219,ENSMUSG00000098222,ENSMUSG00000098224,ENSMUSG00000098232,ENSMUSG00000098234,ENSMUSG00000098235,ENSMUSG00000098238,ENSMUSG00000098240
0,ola_mES_2i_2_1s,6.604476,6.599217,0.0,2.736634,0.000000,7.523991,0.000000,8.728311,0.000000,...,0.0,0.772428,1.272790,1.044193,1.272790,1.938312,6.551002,0.000000,2.065793,3.780998
1,ola_mES_2i_2_10s,6.146365,7.551052,0.0,0.000000,0.000000,8.240065,0.250677,8.236439,0.000000,...,0.0,0.650118,0.814807,1.534887,1.870550,2.261625,6.617063,0.000000,1.943541,3.515651
2,ola_mES_2i_2_11s,7.761644,7.565213,0.0,0.000000,0.000000,7.026434,6.602249,7.807084,0.000000,...,0.0,1.040312,1.327418,1.237918,0.930336,0.380626,6.042423,0.380626,0.930336,3.512956
3,ola_mES_2i_2_12s,8.367752,7.161749,0.0,0.000000,0.000000,3.547153,0.000000,8.116923,0.000000,...,0.0,0.370572,0.525366,0.792575,1.608837,1.118667,6.741700,0.000000,1.974036,3.651705
4,ola_mES_2i_2_13s,6.618116,8.185765,0.0,0.000000,0.000000,6.957035,1.468765,8.290717,0.000000,...,0.0,0.813647,0.589859,1.330729,1.330729,1.007338,5.561222,0.324868,1.710590,3.896294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,ola_mES_lif_3_90s,8.842471,5.531304,0.0,0.455104,7.842117,10.402590,6.985600,6.050395,0.000000,...,0.0,1.079050,1.689428,3.307596,3.195252,2.446409,4.394212,0.455104,3.135618,3.411821
700,ola_mES_lif_3_92s,7.327939,6.320029,0.0,4.394841,0.000000,8.590420,2.979593,6.060602,0.000000,...,0.0,0.000000,2.046402,2.730171,2.428422,2.587171,7.383095,2.428422,2.587171,3.541581
701,ola_mES_lif_3_94s,9.534857,5.319054,0.0,0.000000,0.000000,9.611030,2.085209,5.090952,2.085209,...,0.0,1.845869,2.470072,2.470072,2.085209,2.085209,6.918918,0.000000,2.773587,3.237679
702,ola_mES_lif_3_95s,8.695979,6.706378,0.0,0.000000,0.000000,7.498861,0.000000,5.893844,0.000000,...,0.0,0.715819,2.074533,2.279321,3.011956,1.549545,7.359079,0.000000,2.279321,3.122506


In [28]:
adata = ad.read_h5ad('/Volumes/xyao/MICA/scMINER_13datasets_batch_run_MICA/Kolod/Kolod_MICA.log2.h5ad')
adata

AnnData object with n_obs × n_vars = 704 × 13923
    obs: 'cell_id', 'true_label', 'nUMI.total', 'nGene', 'percent.mito', 'percent.spikeIn'
    var: 'newGeneID', 'gene_id', 'gene_name', 'gene_type', 'nCells'

In [29]:
adata.X

array([[6.6044765 , 6.5992174 , 0.        , ..., 0.        , 2.0657926 ,
        3.7809985 ],
       [6.146365  , 7.5510516 , 0.        , ..., 0.        , 1.9435405 ,
        3.5156507 ],
       [7.7616444 , 7.565213  , 0.        , ..., 0.38062572, 0.9303355 ,
        3.5129564 ],
       ...,
       [9.534857  , 5.319054  , 0.        , ..., 0.        , 2.773587  ,
        3.237679  ],
       [8.695979  , 6.706378  , 0.        , ..., 0.        , 2.2793214 ,
        3.1225054 ],
       [8.131225  , 5.328879  , 0.        , ..., 0.        , 3.0062394 ,
        3.3283455 ]], dtype=float32)

In [33]:
adata.var_names

Index(['ENSMUSG00000000001', 'ENSMUSG00000000028', 'ENSMUSG00000000037',
       'ENSMUSG00000000056', 'ENSMUSG00000000058', 'ENSMUSG00000000078',
       'ENSMUSG00000000085', 'ENSMUSG00000000088', 'ENSMUSG00000000094',
       'ENSMUSG00000000120',
       ...
       'ENSMUSG00000098211', 'ENSMUSG00000098217', 'ENSMUSG00000098219',
       'ENSMUSG00000098222', 'ENSMUSG00000098224', 'ENSMUSG00000098232',
       'ENSMUSG00000098234', 'ENSMUSG00000098235', 'ENSMUSG00000098238',
       'ENSMUSG00000098240'],
      dtype='object', length=13923)

In [30]:
# MICA input 
Kolod_MICA_input='/Volumes/projects/scRNASeq/yu3grp/scMINER/NG_Revision/00_DownloadedDatasets/00_rawData/07_Kolod/Kolod_MICA_input.txt'
df = pd.read_table(Kolod_MICA_input)
df

,ID,1,2,3,4,5,6,7,8,9,...,13464,13465,13466,13467,13468,13469,13470,13471,13472,13473
0,1,2.1724,2.1972,0.00000,0.0,0.932050,0.000000,2.4402,0.32831,2.5519,...,3.9409,1.928800,0.73294,0.31499,2.450700,0.00000,0.00000,2.1962,0.63063,1.0444
1,2,2.3096,2.7947,0.00000,0.0,-0.152290,0.000000,2.9266,0.61098,2.7173,...,4.1294,2.185800,1.18350,0.90749,0.085793,0.00000,0.00000,2.5314,0.90420,1.2424
2,3,2.9274,2.9055,0.00000,0.0,-0.152290,0.000000,2.6592,2.96200,2.6762,...,4.3143,1.174800,0.98529,0.95191,0.085793,0.00000,0.00000,2.4441,0.42303,1.3490
3,4,3.1332,2.7890,0.00000,0.0,-0.152290,0.000000,1.5959,0.32831,2.8207,...,4.3445,-0.084487,1.07230,0.97255,3.158000,0.00000,0.00000,2.7050,1.06250,1.4128
4,5,2.3302,2.8671,0.00000,0.0,-0.152290,0.000000,2.4152,1.17630,2.5717,...,3.7913,-0.084487,0.73294,0.85812,0.085793,0.00000,0.00000,2.0170,0.63063,1.2424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,700,3.1098,2.2512,1.93710,0.0,0.378240,2.026500,3.5330,2.29970,2.4879,...,4.2769,-0.039967,1.66420,1.17770,0.319090,1.30900,1.34330,1.8685,1.29370,1.5028
700,701,2.4782,2.2644,0.91922,0.0,1.572900,-0.037846,2.7653,0.88705,2.2945,...,4.0241,-0.039967,1.20160,1.41000,2.333600,0.96873,0.88192,2.4926,0.94132,1.3489
701,702,3.0784,1.9625,-0.27516,0.0,0.082861,-0.037846,3.0554,0.59424,2.0259,...,4.2011,-0.039967,1.25030,1.35770,2.867800,1.04740,0.75625,2.3509,0.98285,1.2536
702,703,2.8489,2.3647,0.69156,0.0,0.082861,-0.037846,2.4276,-0.16264,2.2411,...,4.0490,-0.039967,1.00730,0.92224,0.029651,0.96873,1.05900,2.4759,0.84225,1.2254


In [13]:
# Check MICA output

In [34]:
Kolod_MICA_output = '/Volumes/xyao/MICA/Kolod/clustering_UMAP_euclidean_19_3.txt'
df = pd.read_table(Kolod_MICA_output)
df

,ID,X,Y,label
0,1,13.096816,8.422349,1
1,2,9.186151,8.325988,1
2,3,8.821341,8.430805,1
3,4,12.276936,8.973976,1
4,5,11.822199,9.301785,1
...,...,...,...,...
699,700,-2.024442,-1.305416,2
700,701,-2.167131,-2.126193,2
701,702,-1.885468,-2.246770,2
702,703,-2.579200,-2.748144,2


In [38]:
Kolod_true_label = '/Volumes/xyao/MICA/Kolod/Kolod_true_label.txt'
true_label_df = pd.read_table(Kolod_true_label)
true_label_df = true_label_df.rename(columns={'label':'true_label'})
true_label_df

,cell,true_label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
699,700,1
700,701,1
701,702,1
702,703,1


In [39]:
merge_df = pd.concat([df,true_label_df],axis=1)
merge_df

,ID,X,Y,label,cell,true_label
0,1,13.096816,8.422349,1,1,0
1,2,9.186151,8.325988,1,2,0
2,3,8.821341,8.430805,1,3,0
3,4,12.276936,8.973976,1,4,0
4,5,11.822199,9.301785,1,5,0
...,...,...,...,...,...,...
699,700,-2.024442,-1.305416,2,700,1
700,701,-2.167131,-2.126193,2,701,1
701,702,-1.885468,-2.246770,2,702,1
702,703,-2.579200,-2.748144,2,703,1


In [40]:
np.round(adjusted_rand_score(merge_df['true_label'],merge_df['label']),3)

1.0

In [15]:
Kolod_MICA_output = './Kolod/Kolod_component_12_binsize_10/clustering_UMAP_euclidean_12_3.txt'
df = pd.read_table(Kolod_MICA_output)
df

,ID,X,Y,label
0,0,13.157156,19.718826,1
1,1,12.394119,20.344206,1
2,2,12.217865,20.213460,1
3,3,13.383860,18.505938,1
4,4,13.441978,18.251469,1
...,...,...,...,...
699,699,-6.046468,-2.608786,2
700,700,-5.167758,-2.879722,2
701,701,-5.423323,-2.682700,2
702,702,-4.754110,-2.771973,2


In [16]:
Kolod_true_label = '../../../../Dataset/scMINER_13datasets_h5ad_input_true_label/Kolod_true_label.txt'
true_df = pd.read_table(Kolod_true_label)
true_df

,cell,true_label
0,ola_mES_2i_2_1s,1
1,ola_mES_2i_2_10s,1
2,ola_mES_2i_2_11s,1
3,ola_mES_2i_2_12s,1
4,ola_mES_2i_2_13s,1
...,...,...
699,ola_mES_lif_3_90s,3
700,ola_mES_lif_3_92s,3
701,ola_mES_lif_3_94s,3
702,ola_mES_lif_3_95s,3


In [21]:
adata = ad.read_text(Kolod_MICA_input, delimiter='\t', first_column_names=True)

In [22]:
adata.obs

""
ola_mES_2i_2_1s
ola_mES_2i_2_10s
ola_mES_2i_2_11s
ola_mES_2i_2_12s
ola_mES_2i_2_13s
...
ola_mES_lif_3_90s
ola_mES_lif_3_92s
ola_mES_lif_3_94s
ola_mES_lif_3_95s
